# Compare Gen methods

In [ ]:
from plm_model import *
import numpy as np
import torch
import os
import sys
#current_dir = os.path.dirname(__file__)
current_dir = os.getcwd()

parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.insert(0, parent_dir)
from model import AttentionModel
from attention import trainer
from dcascore import *
# back to original path (in PLM)
sys.path.pop(0)  # Removes the parent_dir from sys.path
from plm_methods import read_tensor_from_txt, set_seed, letters_to_nums, modify_seq

## Load Q, K, V; compute J

In [ ]:
##############################################################
"""
    Load Q, K, V matrices from jdoms (after training)
"""
set_seed()
H = 64
d= 10
N = 174
n_epochs = 500
loss_type = 'without_J'
family = 'jdoms' #'jdoms_bacteria_train2'
cwd = '/Users/marzioformica/Desktop/EPFL/Master/MA2/Labo/my_project/PLM-gen-DCA/Attention-DCA-main/CODE/AttentionDCA_python/src'
Q_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_youss/Q_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
K_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_youss/K_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
V_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_youss/V_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
H,d,N=Q_1.shape
q=V_1.shape[1]

##############################################################
"""
    Initialize the model and compute couplings J from Q, K, V
""" 
model=AttentionModel(H,d,N,q,Q=Q_1,V=V_1,K=K_1)
torch.sum(model.Q-Q_1)
device = Q_1.device
L = Q_1.shape[-1]
W=attention_heads_from_model(model,Q_1,K_1,V_1)
print(W.shape)

i_indices = torch.arange(L, device=device).unsqueeze(1)
j_indices = torch.arange(L, device=device).unsqueeze(0)
mask = (i_indices != j_indices).float().unsqueeze(0)  # shape (1, L, L)
W = W * mask
    
# Compute Jtens
Jtens = torch.einsum('hri,hab->abri', W, V_1)  # Shape: (q, q, L, L)
q = Jtens.shape[0]
N = Jtens.shape[2]
print(q)
print(N)

torch.Size([64, 63, 63])
21
63


In [ ]:
#print first line of Jtens
print(Jtens[:,:,30,31])

tensor([[ 8.4190e-02, -1.1062e-03, -7.9344e-02, -6.0122e-03,  1.8884e-02,
          3.7376e-02, -6.7353e-03, -2.6382e-02,  1.9143e-02,  9.7130e-02,
         -4.3858e-02, -3.0229e-02,  4.0515e-02, -1.2474e-02,  5.2964e-02,
         -6.5531e-02,  6.6707e-03,  2.4711e-02, -1.1974e-02, -1.7580e-02,
          2.6321e-02],
        [-1.9488e-02, -2.1032e-04, -2.5435e-02,  5.5132e-03, -1.3635e-02,
         -8.1761e-03,  3.0495e-03, -1.3606e-02, -1.4065e-02, -4.8213e-02,
         -4.1118e-03, -8.1658e-03,  4.1129e-03,  1.3696e-03, -1.2804e-02,
         -2.0105e-02, -7.4580e-03, -1.5246e-02, -4.5735e-03,  1.1006e-02,
         -2.0161e-03],
        [-3.4951e-02,  5.6149e-03, -9.6376e-02, -8.7806e-02,  4.6556e-02,
         -1.1157e-02, -9.2960e-03, -7.7389e-04,  1.4078e-01, -4.5730e-02,
         -2.6454e-03,  1.3194e-02, -1.9140e-02,  2.7268e-02,  1.5225e-01,
          5.0871e-02, -1.6868e-02, -1.0864e-02, -2.9854e-03, -8.3227e-02,
          1.3813e-04],
        [-2.4088e-02,  1.1599e-02, -3.6280e

In [ ]:
N_seqs = 5000
init_seq = 'DYYQVLGVPKDADAKSIKKAFRKLARKYHPDVNPGDKEAERKFKEANEANEVLSDPEKRKKYD'
init_sequence_num = letters_to_nums(init_seq)
ratio = 0.1
init_sequence_num = modify_seq(init_sequence_num, ratio)

In [ ]:
from plm_gen_methods import generate_plm
gen_sequences_M = generate_plm(Jtens, N_seqs, init_sequence_num)

100%|██████████| 5000/5000 [01:07<00:00, 74.23it/s]


In [ ]:
from plm_model import generate_plm_y
print(init_sequence_num)
gen_sequences_y=generate_plm_y(Jtens,initial_seq=init_sequence_num,maxiter=2000,beta=1)

[ 2 19 19 11 17  9  5 17  0 13  5  0  2  2  2  3  7 14 14  0 19 14  8  9
  0  8 14 19  6 12  2  8 15 20 12  3 12  8  0  3  3 14  4  8  3 17 15  3
  0 11  3 17  9  5  2 12  5  8 14  0 18 19  2]


100%|██████████| 2000/2000 [01:01<00:00, 32.46it/s]


In [ ]:
from plm_methods import nums_to_letters
for i in range(0, len(gen_sequences_M), 100):
    # Convert the sequence at index i to letters and print
    sequence_in_letters = nums_to_letters(gen_sequences_M[i])
    print(f"Sequence {i + 1} (in letters): {sequence_in_letters}")

Sequence 1 (in letters): YYYWVLGVPKDADAKSIKKAFRKMARKYHPDVNPGDKEAERKFKEANIANEVLLDPEKRFKYD
Sequence 101 (in letters): DYYKVLGVPEDVSEEDIRKAYRKLAIKYHPDRHPHDKEADEKFQSANEAYEVLSDEEKRKQYD
Sequence 201 (in letters): DFYDVLGVSRDASEDEIKKAYRLLAKKYHPDATKDDETAAHAFKEIATAYRVLSDSGPRQKYD
Sequence 301 (in letters): -LHDVLGLSQGASTEEVKLAPRLLAKQLHPDAN-GDNDAATFFDEVNEAYSILSDPEERRRYD
Sequence 401 (in letters): NLYEVLGLAQGASTEEVKKAYRKKAMRLHPDANPPNNEAETRFTEVNGAYESLSDPELRKRYD
Sequence 501 (in letters): NPYEVLGVRQGADEKQIKSAYRQLAQQLHPDANPGNAYAEERFKEVNIAYEVLNSPKKRQRYD
Sequence 601 (in letters): DHYAALQVRRDAEPKQIRSAFRRLAQELHPDLNPSPKQAEERFKALSAAYEILSNAERRARFD
Sequence 701 (in letters): DHYAMLQASLDAEREDVKKAYRKLSRELHPELKPESAAAEDRFKALSTAYEVLSDAARRAEHD
Sequence 801 (in letters): DHYQTLGIAPTAEIQTVRSAYRRLAAEFHPERNPERQAAPDRFVAITAAYEVLSDPRRRAEYD
Sequence 901 (in letters): DHYDDLEISPTASIYQIRQAYFRQAQLYHPERH-KAPIARDVFAAIQHAYAVLSDPSQRSAYD
Sequence 1001 (in letters): DHYQTLGVSVGASAEQIRQAFKRLALRNHPDRLVDAPGAREVIARVQTAYEVLADHERRQRHD


In [ ]:
for i in range(0, len(gen_sequences_y), 100):
    # Convert the sequence at index i to letters and print
    sequence_in_letters = nums_to_letters(gen_sequences_y[i])
    print(f"Sequence {i + 1} (in letters): {sequence_in_letters}")

Sequence 1 (in letters): DYYNVLGVAQGADDDEIRRAYRKLAKRYHPDKS-PEPKAEERFKEVSEANEVLGDPGKRAWYD
Sequence 101 (in letters): DYYEVLGVDQGADEKEIKKAYRKLAKKYHPDVN-KDPGAEEKFKEISEAYDVLSDPEKRAMYD
Sequence 201 (in letters): DYYEILGVSQDASEAEIKKAYRKLARKYHPDVN-KDPQAEEKFKEISEAYEVLSDPEKRAMYD
Sequence 301 (in letters): DYYEILGVSRSASAEEIKKAYRKLARKYHPDLNPSDPEAEEKFKEISEAYEVLSDPDKRALYD
Sequence 401 (in letters): DYYEILGVSKKASEEEIKKAYRKLAKKYHPDLNPSDKEAEEKFKEINEAYEVLSDKEKRALYD
Sequence 501 (in letters): DYYKILGVSKNASEDEIKKAYRKLAKKYHPDLNPSDKEAEEKFKEIKEAYEVLSDPKKRAQYD
Sequence 601 (in letters): DYYEILGVSKSASEDDIKKAYRKLAKKYHPDKNPSDKEAEEKFKEISEAYEVLSDPQKRKRYD
Sequence 701 (in letters): DYYEILGVSKHASAEEIKKAYRKLAMKYHPDRNPGNKEAEEKFKEISEAYEILSDEEKRKRYD
Sequence 801 (in letters): DYYEILGVSKDASMDEIKKAYRKLARKYHPDLNPDDPEAEAKFKEINEAYEVLSDEEKRARYD
Sequence 901 (in letters): DYYEILGVSKGASTDEIKKAYRKLARKYHPDLNPDDPEAEERFKEISEAYEVLSDPEKRKRYD
Sequence 1001 (in letters): DYYEVLGVTKSASQDEIKKAYRKLALKYHPDRNPGDKEAERKFKEVNEAYAVLSDPEKRAQYD
